In [ ]:
pip install -U huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
 !apt-get install git-lfs


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
# imports
import pandas as pd
import numpy as np
import pickle
from transformers import pipeline
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from torch.utils.data import Dataset
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
# Load model directly
tokenizer = AutoTokenizer.from_pretrained("ElizaClaPa/SentimentAnalysis-YelpReviews-OptimizedModel")
model = AutoModelForSequenceClassification.from_pretrained("ElizaClaPa/SentimentAnalysis-YelpReviews-OptimizedModel")

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
# setting up the datasets for the Trainer

In [ ]:
# split dataset filepaths
ds_train_filepath = '/content/drive/My Drive/LLMproject/ds_train.csv'
ds_test_filepath = '/content/drive/My Drive/LLMproject/ds_test.csv'

In [ ]:
# loading dataset split
ds_train = pd.read_csv(ds_train_filepath)
ds_test = pd.read_csv(ds_test_filepath)

In [ ]:
# preprocessed filepaths
train_preprocessed_removed_nulls_filepath = '/content/drive/My Drive/LLMproject/train_preprocessed_removed_nulls_.csv'
test_preprocessed_removed_nulls_filepath = '/content/drive/My Drive/LLMproject/test_preprocessed_removed_nulls_.csv'

In [ ]:
# loading preprocessed
train_preprocessed_removed_nulls = pd.read_csv(train_preprocessed_removed_nulls_filepath)
test_preprocessed_removed_nulls = pd.read_csv(test_preprocessed_removed_nulls_filepath)

In [ ]:
# getting a sample of training and testing data to account for computing limits

In [ ]:
# training sample
train_labeled = train_preprocessed_removed_nulls.join(ds_train['label'])
train_sample = train_labeled['preprocessed_text'][:100]
train_sample_text = train_sample.tolist()
train_sample_encodings = tokenizer(train_sample_text, truncation=True, padding=True)

In [ ]:
# testing sample
test_labeled = test_preprocessed_removed_nulls.join(ds_test['label'])
test_sample = test_labeled['preprocessed_text'][:50]
test_sample_text = test_sample.tolist()
test_sample_encodings = tokenizer(test_sample_text, truncation=True, padding=True)

In [ ]:
# sample filepaths
train_sample_filepath = '/content/drive/My Drive/LLMproject/train_sample.csv'
test_sample_filepath = '/content/drive/My Drive/LLMproject/test_sample.csv'

In [ ]:
# loading samples
train_sample = pd.read_csv(train_sample_filepath)
test_sample = pd.read_csv(test_sample_filepath)

In [ ]:
# getting labels for dataset
train_labels = train_sample['label'].tolist()
test_labels = test_sample['label'].tolist()

In [ ]:
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# Create datasets
train_dataset = SentimentDataset(train_sample_encodings, train_labels)
test_dataset = SentimentDataset(test_sample_encodings, test_labels)

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/drive/My Drive/LLMproject/sentiment_analysis_transfer_learning',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=7, training_loss=0.8075902802603585, metrics={'train_runtime': 388.357, 'train_samples_per_second': 0.257, 'train_steps_per_second': 0.018, 'total_flos': 12626474424000.0, 'train_loss': 0.8075902802603585, 'epoch': 1.0})

In [ ]:
# saving the trained model
trained_model_filepath = '/content/drive/My Drive/LLMproject/trained_model'

In [ ]:
trainer.save_model(trained_model_filepath)

In [ ]:
# loading in model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(trained_model_filepath)
tokenizer = AutoTokenizer.from_pretrained(trained_model_filepath)

In [ ]:
repo_name = 'KelseyHahmo/Yelp-Review-Sentiment-Analysis'
training_args = TrainingArguments(
   output_dir=repo_name,
   push_to_hub=True,
   learning_rate=2e-5, # lower rate for smoother adjustment
   num_train_epochs=3, # don't want to overfit so using a lower number
   weight_decay=0.01 # want to avoid overfitting while not compromising on model strength
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=39, training_loss=0.5017368610088642, metrics={'train_runtime': 1082.5862, 'train_samples_per_second': 0.277, 'train_steps_per_second': 0.036, 'total_flos': 37879423272000.0, 'train_loss': 0.5017368610088642, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()


CommitInfo(commit_url='https://huggingface.co/KelseyHahmo/Yelp-Review-Sentiment-Analysis/commit/1dce630a4742eb00d445ee928cea5db48b9541ed', commit_message='End of training', commit_description='', oid='1dce630a4742eb00d445ee928cea5db48b9541ed', pr_url=None, repo_url=RepoUrl('https://huggingface.co/KelseyHahmo/Yelp-Review-Sentiment-Analysis', endpoint='https://huggingface.co', repo_type='model', repo_id='KelseyHahmo/Yelp-Review-Sentiment-Analysis'), pr_revision=None, pr_num=None)

In [ ]:
data = test_preprocessed_removed_nulls['preprocessed_text'][51:100].tolist()
my_model = pipeline(model='KelseyHahmo/Yelp-Review-Sentiment-Analysis')
predictions = my_model(data)

In [ ]:
predictions

[{'label': '3 stars', 'score': 0.9103151559829712},
 {'label': '3 stars', 'score': 0.7423199415206909},
 {'label': '2 star', 'score': 0.6264269351959229},
 {'label': '3 stars', 'score': 0.8994715809822083},
 {'label': '5 stars', 'score': 0.6693228483200073},
 {'label': '3 stars', 'score': 0.7639105319976807},
 {'label': '2 star', 'score': 0.915913462638855},
 {'label': '4 stars', 'score': 0.4785011112689972},
 {'label': '2 star', 'score': 0.8698422312736511},
 {'label': '5 stars', 'score': 0.9398338198661804},
 {'label': '1 star', 'score': 0.9369410276412964},
 {'label': '1 star', 'score': 0.9959797859191895},
 {'label': '4 stars', 'score': 0.6879689693450928},
 {'label': '5 stars', 'score': 0.9397622346878052},
 {'label': '5 stars', 'score': 0.943081796169281},
 {'label': '1 star', 'score': 0.8881489634513855},
 {'label': '4 stars', 'score': 0.679387629032135},
 {'label': '2 star', 'score': 0.6701962947845459},
 {'label': '3 stars', 'score': 0.8377468585968018},
 {'label': '1 star', '

In [ ]:
label_mapping = {
    '1 star': 1,
    '2 star': 2,
    '3 stars': 3,
    '4 stars': 4,
    '5 stars': 5
}

# Convert predicted labels to integers using the mapping
predicted_labels = [label_mapping[prediction['label']] for prediction in predictions]
predicted_labels

[3,
 3,
 2,
 3,
 5,
 3,
 2,
 4,
 2,
 5,
 1,
 1,
 4,
 5,
 5,
 1,
 4,
 2,
 3,
 1,
 2,
 5,
 4,
 5,
 4,
 1,
 4,
 3,
 3,
 2,
 1,
 2,
 5,
 3,
 1,
 5,
 5,
 2,
 2,
 3,
 5,
 3,
 3,
 4,
 3,
 2,
 4,
 1,
 5]

In [ ]:
#
true_labels = test_labeled['label'][51:100]
true_labels.head()

,label
51,2
52,2
53,0
54,3
55,3


In [ ]:
# comparing with actual star ratings
accuracy = accuracy_score(true_labels, predicted_labels)
accuracy

0.12244897959183673

In [ ]:
report = classification_report(true_labels, predicted_labels, zero_division=1)
report

'              precision    recall  f1-score   support\n\n           0       1.00      0.00      0.00        10\n           1       0.25      0.15      0.19        13\n           2       0.00      0.00      0.00         5\n           3       0.17      0.18      0.17        11\n           4       0.25      0.20      0.22        10\n           5       0.00      1.00      0.00         0\n\n    accuracy                           0.12        49\n   macro avg       0.28      0.26      0.10        49\nweighted avg       0.36      0.12      0.13        49\n'

In [ ]:
# more training data would help this. I wasn't able to use a lot of training data due to limits on RAM

In [ ]:
# the amount I used was very close to hitting my limit

In [ ]:
drive.flush_and_unmount()